In [ ]:
import time

import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pandas as pd 
import seaborn as sns
import cv2

In [ ]:
train_images = np.load("Kannada_MNIST_npz/Kannada_MNIST/X_kannada_MNIST_train.npz")['arr_0']
test_images = np.load("Kannada_MNIST_npz/Kannada_MNIST/X_kannada_MNIST_test.npz")['arr_0']
train_labels = np.load("Kannada_MNIST_npz/Kannada_MNIST/y_kannada_MNIST_train.npz")['arr_0']
test_labels = np.load("Kannada_MNIST_npz/Kannada_MNIST/y_kannada_MNIST_test.npz")['arr_0']
digi_images = np.load("Kannada_MNIST_npz/Dig_MNIST/X_dig_MNIST.npz")['arr_0']
digi_labels = np.load("Kannada_MNIST_npz/Dig_MNIST/y_dig_MNIST.npz")['arr_0']

In [ ]:
thresh = 83.9967
bin_train_images = cv2.threshold(train_images, thresh, 255, cv2.THRESH_BINARY)[1]
bin_test_images = cv2.threshold(test_images, thresh, 255, cv2.THRESH_BINARY)[1]
bin_digi_images = cv2.threshold(digi_images, thresh, 255, cv2.THRESH_BINARY)[1]

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(bin_train_images[i], cmap=plt.cm.binary)
    plt.xlabel(train_labels[i])

In [ ]:
import pywt
coeffs_train = pywt.dwt2(bin_train_images, 'haar')
coeffs_test = pywt.dwt2(bin_test_images, 'haar')
coeffs_digi = pywt.dwt2(bin_digi_images, 'haar')
cA_train, (cH_train, cV_train, cD_train) = coeffs_train
cA_test, (cH_test, cV_test, cD_test) = coeffs_test
cA_digi, (cH_digi, cV_digi, cD_digi) = coeffs_digi

In [ ]:
from scipy.fftpack import dct as dct
dct_train = dct(cA_train)
dct_test = dct(cA_test)
dct_digi = dct(cA_digi)

In [ ]:
dct_train = dct_train.reshape(60000, 14*14)
dct_test = dct_test.reshape(10000, 14*14)
dct_digi = dct_digi.reshape(10240, 14*14)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
mscaler = MinMaxScaler()
x_train = mscaler.fit_transform(dct_train)#np.expand_dims(dct_train, axis=-1)
x_test = mscaler.fit_transform(dct_test)#np.expand_dims(dct_test, axis=-1)
x_digi_test = mscaler.fit_transform(dct_digi)#np.expand_dims(dct_digi, axis=-1)

y_train = train_labels#keras.utils.to_categorical(train_labels)
y_test = test_labels#keras.utils.to_categorical(test_labels)
y_digi_test = digi_labels#keras.utils.to_categorical(digi_labels)

In [ ]:
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='sigmoid') # Linear Kernel

#Train the model using the training sets
clf.fit(x_train, y_train)

#Predict the response for test dataset
y_test_pred = clf.predict(x_test)
y_digi_pred = clf.predict(x_digi_test)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
def plot_confusion_matrix(confusion_matrix, y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix#confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes#[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(figsize=(8, 8))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#pred = model.predict(x_test, batch_size=32, verbose=1)
predicted = y_test_pred#np.argmax(pred, axis=1)
report = classification_report(y_test, predicted, digits = 4)
confusion_matrix = confusion_matrix(y_true=y_test, y_pred=predicted)
print(report)
plot_confusion_matrix(confusion_matrix,y_true=y_test, y_pred=predicted, classes=['0','1','2','3','4','5','6','7','8','9'],
                      title='Confusion matrix, without normalization')

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

#pred = model.predict(x_digi_test, batch_size=32, verbose=1)
predicted = y_digi_pred#np.argmax(pred, axis=1)
report = classification_report(y_digi_test, predicted, digits = 4)
confusion_matrix = confusion_matrix(y_true=y_digi_test, y_pred=predicted)
print(report)
plot_confusion_matrix(confusion_matrix,y_true=y_digi_test, y_pred=predicted, classes=['0','1','2','3','4','5','6','7','8','9'],
                      title='Confusion matrix, without normalization')